# Raw Neural Net

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
import numpy
import scipy.special

In [3]:
class NeuralNetwork():
    
    # initialize NeuralNetwork
    def __init__(self, n_inodes, n_hnodes, n_onodes, lrate):
        # set the number of nodes in each layer
        self.n_inodes = n_inodes
        self.n_hnodes = n_hnodes
        self.n_onodes = n_onodes
        
        # learning rate
        self.lr = lrate
        
        ''' weight initialization, by np.rand 
            np.rand(rows, columns) - 0.5 to center from +/- 0.5
        '''
        self.w_i2h = (numpy.random.rand(self.n_hnodes, self.n_inodes) - 0.5)
        self.w_h2o = (numpy.random.rand(self.n_onodes, self.n_hnodes) - 0.5)
        
        ''' weight initialization, by np.normal 
                (selecting from a normal distribution)
            np.normal(center, stdev, (rows, columns))
        '''
        self.w_i2h = numpy.random.normal(0.0, pow(self.n_hnodes, -0.5), (self.n_hnodes, self.n_inodes))
        self.w_h2o = numpy.random.normal(0.0, pow(self.n_onodes, -0.5), (self.n_onodes, self.n_hnodes))
                                         
        self.activation = lambda x: scipy.special.expit(x)
                                         
        pass
    
    # train NeuralNetwork
    def train(self, input_list, target_list):
        
        inputs  = numpy.array(input_list, ndmin=2).T
        targets = numpy.array(target_list, ndmin=2).T
        
        hidden_in  = numpy.dot(self.w_i2h, inputs)
        hidden_out = self.activation(hidden_in)
        
        output_in  = numpy.dot(self.w_h2o, hidden_out)
        output_out = self.activation(output_in)
        
        output_err = targets - output_out
        
        hidden_err = numpy.dot(self.w_h2o.T, output_err)
        
        self.w_h2o += self.lr * numpy.dot((output_err * output_out * (1 - output_out)), numpy.transpose(hidden_out))
        
        self.w_i2h += self.lr * numpy.dot((hidden_err * hidden_out * (1 - hidden_out)), numpy.transpose(inputs))

        pass
    
    # query NeuralNetwork
    def query(self, input_list):
        inputs = numpy.array(input_list, ndmin=2).T
                                     
        # X_hidden = W_{input_hidden} . I
        hidden_in  = numpy.dot(self.w_i2h, inputs)
        # pass `hidden_in` through the activation function to calculate the output
        hidden_out = self.activation(hidden_in)
            
        # X_output = W_{hidden_output} . I_{hidden_out}                                
        output_in  = numpy.dot(self.w_h2o, hidden_out) 
         # pass `final_in` through the activation function to calculate the output
        output_out = self.activation(output_in)
        
        return output_out

In [4]:
# number of input, hidden, & output nodes
n_inodes = 3
n_hnodes = 3
n_onodes = 3

# learning rate
lrate = 0.3

# create an instance of NeuralNetwork
nn = NeuralNetwork(n_inodes, n_hnodes, n_onodes, lrate)

In [5]:
nn.query([1.0, 0.5, -1.5])

array([[ 0.46619464],
       [ 0.31847565],
       [ 0.55472166]])

## Applied to MNIST

In [6]:
# number of input, hidden, & output nodes
n_inodes = 784
n_hnodes = 100
n_onodes = 10

# learning rate
lrate = 0.1

# create an instance of NeuralNetwork
nn = NeuralNetwork(n_inodes, n_hnodes, n_onodes, lrate)

In [7]:
training_data_file = open("MNIST_data/mnist_train.csv", "r")
training_data_list = training_data_file.readlines()
training_data_file.close()

In [14]:
epochs = 5

for e in range(epochs):
    for record in training_data_list:
        all_values = record.split(",")
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        targets = numpy.zeros(n_onodes) + 0.01
        targets[int(all_values[0])] = 0.99
        print(targets)
        break
        nn.train(inputs, targets)
        pass
    pass

[ 0.01  0.01  0.01  0.01  0.01  0.99  0.01  0.01  0.01  0.01]
[ 0.01  0.01  0.01  0.01  0.01  0.99  0.01  0.01  0.01  0.01]
[ 0.01  0.01  0.01  0.01  0.01  0.99  0.01  0.01  0.01  0.01]
[ 0.01  0.01  0.01  0.01  0.01  0.99  0.01  0.01  0.01  0.01]
[ 0.01  0.01  0.01  0.01  0.01  0.99  0.01  0.01  0.01  0.01]


### Scoring the Network

In [9]:
test_data_file = open("MNIST_data/mnist_test.csv", "r")
test_data_list = test_data_file.readlines()
test_data_file.close()

In [13]:
scorecard = []

for record in test_data_list:
    all_values = record.split(",")
    correct_label = int(all_values[0])
    
    inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    
    outputs = nn.query(inputs)
    
    label = numpy.argmax(outputs)
    
    scorecard.append(1 if label == correct_label else 0)
    print(label)
    print(correct_label)
    print(label == correct_label)
    break
    
    pass
pass

7
7
True


In [11]:
scorecard_np = numpy.asarray(scorecard)
print("Performace = {}".format(scorecard_np.sum() / scorecard_np.size))

Performace = 1.0


In [12]:
%%time

import gzip

def convert(imgf, labelf, outf, n):
    f = gzip.open(imgf, "rb")
    o = open(outf, "w")
    l = gzip.open(labelf, "rb")

    f.read(16)
    l.read(8)
    images = []

    for i in range(n):
        image = [ord(l.read(1))]
        for j in range(28*28):
            image.append(ord(f.read(1)))
        images.append(image)

    for image in images:
        o.write(",".join(str(pix) for pix in image)+"\n")
    f.close()
    o.close()
    l.close()

    
# Extracting MNIST_data/train-images-idx3-ubyte.gz
# Extracting MNIST_data/train-labels-idx1-ubyte.gz
# Extracting MNIST_data/t10k-images-idx3-ubyte.gz
# Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
convert("MNIST_data/train-images-idx3-ubyte.gz", "MNIST_data/train-labels-idx1-ubyte.gz",
        "MNIST_data/mnist_train.csv", 60000)
convert("MNIST_data/t10k-images-idx3-ubyte.gz", "MNIST_data/t10k-labels-idx1-ubyte.gz",
        "MNIST_data/mnist_test.csv", 10000)

CPU times: user 1min 37s, sys: 2.06 s, total: 1min 39s
Wall time: 1min 45s
